# What is a Unified Margin & Coin-Margin Arbitrage Bot? (Simple Explanation)

Imagine you’re at a market where apples are sold at two different stalls.

- Stall A sells apples for $1.00 each
- Stall B sells apples for $1.10 each

If you buy from Stall A and immediately sell at Stall B, you make $0.10 profit per apple.

This is arbitrage—buying low in one place and selling high in another.

# How This Works in Crypto

Instead of apples, we’re trading Bitcoin (BTC) on Binance.
There are two different futures markets for Bitcoin:

1) Unified Margin Futures (USDT-M Futures)

- Uses USDT (Tether) as margin (like paying with cash).

- Example: BTCUSDT Perpetual Futures.

2) Coin-Margin Futures (COIN-M Futures)

- Uses BTC as margin (like trading apples for apples).

- Example: BTCUSD Perpetual Futures.

Since these two markets work differently, prices are sometimes not the same, creating an opportunity to make a profit.

# How the Arbitrage Bot Makes Money

The bot looks for price differences between these two markets:

✅ If BTC is cheaper in Unified Margin (UM) than Coin-Margin (CM):

- Buy in UM (USDT-M Futures) (get BTC at a lower price).
- Sell in CM (COIN-M Futures) (sell BTC at a higher price).
- Profit = CM Price - UM Price

✅ If BTC is cheaper in Coin-Margin (CM) than Unified Margin (UM):

- Buy in CM (COIN-M Futures).
- Sell in UM (USDT-M Futures).
- Profit = UM Price - CM Price.


In [ ]:
fee_rate = 0.001  # 0.1% fee rate
trade_size = 0.01  # btc

# Backtesting the Arbitrage Strategy
def backtest_arbitrage(df):
    initial_balance = 1000  # Start with $1000
    balance = initial_balance
    position = 0  # No initial position
    transaction_history = []

    for i in range(1, len(df)):
        spot_price = df.loc[i, 'close_spot']
        um_futures_price = df.loc[i, 'close_um']
        cm_futures_price = df.loc[i, 'close_cm']

        # Calculate arbitrage opportunities
        um_arbitrage_profit = um_futures_price - spot_price
        cm_arbitrage_profit = cm_futures_price - spot_price

        # Simulate arbitrage trades
        if um_arbitrage_profit > fee_rate:  # If there's a profit in UM arbitrage
            if position == 0:  # No active position
                position = trade_size  # Buy on spot market
                balance -= spot_price * trade_size  # Deduct the capital
                transaction_history.append(('buy', 'spot', spot_price, trade_size))

            # Sell on Unified Margin Futures
            balance += um_futures_price * trade_size - (um_futures_price * trade_size * fee_rate)  # Deduct fees
            position = 0  # Close the position
            transaction_history.append(('sell', 'UM Futures', um_futures_price, trade_size))

        elif cm_arbitrage_profit > fee_rate:  # If there's a profit in CM arbitrage
            if position == 0:  # No active position
                position = trade_size  # Buy on spot market
                balance -= spot_price * trade_size  # Deduct the capital
                transaction_history.append(('buy', 'spot', spot_price, trade_size))

            # Sell on Coin-Margin Futures
            balance += cm_futures_price * trade_size - (cm_futures_price * trade_size * fee_rate)  # Deduct fees
            position = 0  # Close the position
            transaction_history.append(('sell', 'CM Futures', cm_futures_price, trade_size))

        # Output the balance after each trade (for tracking)
        print(f"Timestamp: {df.loc[i, 'timestamp']}, Balance: {balance:.2f}, Position: {position}")

    # Return the final balance and transaction history
    return balance, transaction_history

# Plotting the results
def plot_results(transaction_history):
    # Convert the transaction history into a DataFrame for easy plotting
    trans_df = pd.DataFrame(transaction_history, columns=['action', 'market', 'price', 'size'])
    
    # Plot the profit over time
    plt.figure(figsize=(10,6))
    plt.plot(trans_df.index, trans_df['price'], label='Price', color='blue')
    plt.title('Arbitrage Strategy Performance')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Run the backtest
historical_data = fetch_historical_data()
final_balance, trans_history = backtest_arbitrage(historical_data)

# Print the final balance after backtesting
print(f"Final Balance: ${final_balance:.2f}")
plot_results(trans_history)